<a href="https://colab.research.google.com/github/Sreerag-Pillai/Data_Science/blob/main/Trading_indicators.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook demonstrates the calculation and visualization of various technical indicators commonly used in financial analysis. These indicators include:

Double Exponential Moving Average (DEMA): A trend-following indicator that aims to reduce lag compared to traditional moving averages.

Adaptive Moving Average (AMA) / Kaufman's Adaptive Moving Average (KAMA): An indicator that adjusts its sensitivity based on market volatility.

Moving Average with Variable Period (MAVP): A moving average that adapts its period based on market conditions.

Absolute Price Oscillator (APO): An oscillator that measures the difference between two exponential moving averages of different periods.

Percentage Price Oscillator (PPO): Similar to APO, but expresses the difference as a percentage of the slower moving average.

The notebook provides Python code for calculating these indicators and uses the mplfinance library to create candlestick charts with the indicators overlaid. This allows for visual analysis of price trends and potential trading signals.

In [ ]:
!pip install matplotlib
!pip install pandas
!pip install mplfinance

## Double Exponential Moving Average (DEMA)

### Date: September 2023

#### Available Parameters:
* Period (Default value: 14)
* Single exponential moving average (Single EMA)

source: https://www.investopedia.com/terms/d/double-exponential-moving-average.asp

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import mplfinance as mpf

# Reading the CSV file
data = pd.read_csv("SampleData.csv")

# Convert 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

def calculate_ema(prices, period):
    #Calculate Exponential Moving Average
    ema = prices.ewm(span=period, adjust=False).mean()
    return ema

def calculate_dema(prices, period=15):
    #Calculate Double Exponential Moving Average
    ema = calculate_ema(prices, period)
    ema_of_ema = calculate_ema(ema, period)
    dema = 2 * ema - ema_of_ema
    return dema

# Calculate DEMA for the Close prices
data['DEMA'] = calculate_dema(data['Close'])

# Function to plot OHLC candlestick chart with DEMA
def plot_candlestick_indicator(data, indicator_column, title):
    ap = [mpf.make_addplot(data[indicator_column], color='r')]
    mpf.plot(data, type='candle', style='yahoo', title=title,
             volume=False, addplot=ap, figratio=(14, 7), ylabel='Price')

# Plot OHLC candlestick chart with DEMA
plot_candlestick_indicator(data, 'DEMA', 'Stock Price with Double Exponential Moving Average (DEMA)')


## Adaptive Moving Average (AMA) - Also known as Kaufman's Adaptive Moving Average (KAMA)

### Date: September 2023

#### Available Parameters:
* Efficiency Ratio (ER)(period: 10)
* Fast EMA (Default value: 2)
* Slow EMA (Default value: 30)

source:https://school.stockcharts.com/doku.php?id=technical_indicators:kaufman_s_adaptive_moving_average

In [ ]:
# Reading the CSV file
data = pd.read_csv("SampleData.csv")

# Convert 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# KAMA (Adaptive Moving Average)
def calculate_ama(prices, period=10, fast_ema=2, slow_ema=30):
    change = prices.diff(period).abs()
    volatility = prices.diff().abs().rolling(period).sum()
    volatility = volatility.where(volatility != 0, 1)
    ER = change / volatility
    fastest_sc = 2 / (fast_ema + 1)
    slowest_sc = 2 / (slow_ema + 1)
    SC = ER * (fastest_sc - slowest_sc) + slowest_sc
    SC_squared = SC * SC
    AMA = prices.copy()
    for i in range(period, len(prices)):
        AMA[i] = AMA[i - 1] + SC_squared[i] * (prices[i] - AMA[i - 1])
    return AMA


# Calculate KAMA for the Close prices
data['KAMA'] = calculate_ama(data['Close'])

# Function to plot OHLC candlestick chart with overlayed KAMA
def plot_candlestick_with_overlayed_indicator(data, indicator_column, title):
    ap = [mpf.make_addplot(data[indicator_column], color='r')]
    mpf.plot(data, type='candle', style='yahoo', title=title,
             volume=False, addplot=ap, figratio=(14, 7), ylabel='Price')

# Plot OHLC candlestick chart with KAMA
plot_candlestick_indicator(data, 'KAMA', 'Stock Price with Kaufman Adaptive Moving Average (KAMA)')


## Moving Average with Variable Period (MAVP)

### Date: September 2023

#### Available Parameters:
* Period (Default value: 14)


source: https://trendspider.com/learning-center/what-is-the-moving-average-with-variable-period-mavp/#:~:text=Moving%20Average%20with%20Variable%20Period%20(MAVP)%20is%20a%20powerful%20technical,the%20accuracy%20of%20trend%20identification.


In [ ]:
# Reading the CSV file
data = pd.read_csv("SampleData.csv")

# Convert 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

def calculate_weighted_mavp(prices, period=14):
    #Calculate Moving Average with Variable Period (MAVP) based on weighted average approach.

    weighted_mavp = []

    for i in range(len(prices)):
        if i < period - 1:
            # Not enough data points to compute MAVP
            weighted_mavp.append(None)
        else:
            weights = list(range(period, 0, -1))  # Generate decreasing weights
            window = prices[i-period+1:i+1]
            mavp_value = sum(window * weights) / sum(weights)
            weighted_mavp.append(mavp_value)

    return pd.Series(weighted_mavp, index=prices.index)

# Calculate MAVP for the Close prices
data['Weighted_MAVP'] = calculate_weighted_mavp(data['Close'], period=14)

# Function to plot OHLC candlestick chart with indicator
def plot_candlestick_with_overlayed_indicator(data, indicator_column, title):
    ap = [mpf.make_addplot(data[indicator_column], color='r')]
    mpf.plot(data, type='candle', style='yahoo', title=title,
             volume=False, addplot=ap, figratio=(14, 7), ylabel='Price')

# Plot the data
plot_candlestick_with_overlayed_indicator(data, 'Weighted_MAVP', 'Stock Price with Moving Average with Variable Period (MAVP)')


## APO - Absolute Price Oscillator
### Date: September 2023

#### Available Parameters:
* Fast EMA (Default value: 12)
* Slow EMA (Default value: 26)

source: https://www.fidelity.com/learning-center/trading-investing/technical-analysis/technical-indicator-guide/apo

In [ ]:
# Reading the CSV file
data = pd.read_csv("SampleData.csv")

# Convert 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# APO
def calculate_apo(prices, fast_ema=12, slow_ema=26):
    fast_ema_values = prices.ewm(span=fast_ema, adjust=False).mean()
    slow_ema_values = prices.ewm(span=slow_ema, adjust=False).mean()
    apo = fast_ema_values - slow_ema_values
    return apo

# Calculate APO and add it to the dataframe
data['APO'] = calculate_apo(data['Close'])

# Plotting
def plot_candlestick_apo(data, apo_column, title):
    ap = [
        mpf.make_addplot(data[apo_column], panel=1, color='r', secondary_y=False),
        mpf.make_addplot([0] * len(data), panel=1, color='black', linestyle='--', secondary_y=False)
    ]
    mpf.plot(data, type='candle', style='yahoo', title=title,
             volume=False, addplot=ap, figratio=(14, 7), ylabel='Price', ylabel_lower='APO')

# Call the plotting function for APO
plot_candlestick_apo(data, 'APO', 'Stock Price with Separate Absolute Price Oscillator (APO)')


## PPO - Percentage Price Oscillator
### Date: September 2023

#### Available Parameters:
* Fast EMA (Default value: 12)
* Slow EMA (Default value: 26)

source: https://www.composer.trade/learn/what-is-percentage-price-oscillator-ppo

In [ ]:
# Reading the CSV file
data = pd.read_csv("SampleData.csv")

# Convert 'Date' column to datetime format and set it as the index
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

# PPO
def calculate_ppo(prices, fast_ema=12, slow_ema=26):
    fast_ema_values = prices.ewm(span=fast_ema, adjust=False).mean()
    slow_ema_values = prices.ewm(span=slow_ema, adjust=False).mean()
    ppo = ((fast_ema_values - slow_ema_values) / slow_ema_values) * 100
    return ppo

# Signal Line
def calculate_signal_line(ppo_values, period=9):
    return ppo_values.ewm(span=period, adjust=False).mean()

# Calculate PPO, Signal Line and Histogram
data['PPO'] = calculate_ppo(data['Close'])
data['PPO_Signal'] = calculate_signal_line(data['PPO'])
data['PPO_Histogram'] = data['PPO'] - data['PPO_Signal']

# Plotting
def plot_candlestick_ppo(data, title):
    ap = [
        mpf.make_addplot(data['PPO'], panel=1, color='blue', secondary_y=False),
        mpf.make_addplot(data['PPO_Signal'], panel=1, color='red', secondary_y=False),
        mpf.make_addplot(data['PPO_Histogram'], panel=1, color='green', type='bar', secondary_y=False)
    ]
    mpf.plot(data, type='candle', style='yahoo', title=title,
             volume=False, addplot=ap, figratio=(14, 7), ylabel='Price', ylabel_lower='PPO')

# Call the plotting function for PPO
plot_candlestick_ppo(data, 'Stock Price with Percentage Price Oscillator (PPO)')
